# 芒果品質分類模型

這個 Jupyter Notebook 會展示如何使用 PyTorch 和預訓練的 ResNet50 模型來訓練一個圖片分類器，以識別不同品質的芒果捏。

## 導入必要的庫

首先，我們需要導入 PyTorch 相關的庫以及 torchvision，後者提供了許多有用的工具和預訓練模型捏。


In [1]:
import torch
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader


## 設定圖片轉換

我們將設定轉換來預處理圖片資料捏。這包括調整圖片大小、轉換為張量以及標準化捏。


In [2]:
# 設定圖片轉換捏
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # 調整圖片大小以適合模型捏
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # 正規化捏
])


## 加載訓練和驗證資料集

使用 ImageFolder 來加載訓練和驗證資料捏，這將自動為我們的圖片標籤化捏。


In [3]:
# 加載訓練和驗證資料捏
train_data = datasets.ImageFolder('dataset/train', transform=transform)
val_data = datasets.ImageFolder('dataset/val', transform=transform)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32, shuffle=False)


## 建立和修改模型

我們將使用預訓練的 ResNet50 模型，並修改最後的全連接層以適應我們的三個類別捏。


In [4]:
model = models.resnet50(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, 3)  # 修改最後一層以匹配三個類別捏


C:\Users\Alan\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\Alan\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


## 設定裝置

確認是否可以使用 CUDA，並設定相應的裝置捏。


In [5]:
if torch.cuda.is_available():
    print("CUDA is available. Training on GPU.")
else:
    print("CUDA is not available. Training on CPU.")

# 設定裝置捏
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)


CUDA is available. Training on GPU.


## 定義損失函數和優化器

為訓練過程設定損失函數和優化器捏。


In [6]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


## 訓練模型

定義一個函數來執行訓練循環，包括前向傳播、損失計算、反向傳播和參數更新捏。


In [7]:
def train_model(model, criterion, optimizer, train_loader, val_loader, device, num_epochs=25):
    model.to(device)
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        model.eval()
        with torch.no_grad():
            correct = 0
            total = 0
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        print(f'Epoch {epoch+1}, Loss: {running_loss / len(train_loader)}, Accuracy: {100 * correct / total}%')


## 開始訓練和儲存模型

最後，我們執行訓練函數並儲存訓練好的模型捏。


In [8]:
# 開始訓練捏
train_model(model, criterion, optimizer, train_loader, val_loader, device)

# 儲存模型捏
torch.save(model.state_dict(), 'mango_classifier.pth')


Epoch 1, Loss: 0.701308946609497, Accuracy: 64.5%
Epoch 2, Loss: 0.599711287021637, Accuracy: 74.75%
Epoch 3, Loss: 0.5704260306698935, Accuracy: 76.75%
Epoch 4, Loss: 0.5143997662408011, Accuracy: 64.0%
Epoch 5, Loss: 0.4912255220753806, Accuracy: 76.375%
Epoch 6, Loss: 0.48826928896563393, Accuracy: 71.625%
Epoch 7, Loss: 0.45617284417152404, Accuracy: 75.875%
Epoch 8, Loss: 0.4399227273464203, Accuracy: 76.75%
Epoch 9, Loss: 0.42387222698756627, Accuracy: 74.25%
Epoch 10, Loss: 0.3729315223012652, Accuracy: 72.0%
Epoch 11, Loss: 0.37132919004985265, Accuracy: 72.0%
Epoch 12, Loss: 0.3030768857257707, Accuracy: 72.875%
Epoch 13, Loss: 0.3016481349297932, Accuracy: 74.5%
Epoch 14, Loss: 0.27622205355337687, Accuracy: 74.625%
Epoch 15, Loss: 0.21128185591527393, Accuracy: 75.75%
Epoch 16, Loss: 0.16629441560379096, Accuracy: 74.125%
Epoch 17, Loss: 0.19859074381845337, Accuracy: 72.5%
Epoch 18, Loss: 0.1510795097798109, Accuracy: 68.125%
Epoch 19, Loss: 0.1597613086551428, Accuracy: 75